Building Financial Model in Python

A Step-by-step guide on how to build a sturdy discounted cash flow model to calculate NPV, IRR, Payback Period and Multiple Invested Capital (MOIC) using Python.

What Is Discounted Cash Flow (DCF) Model ? How does it help?
Discounted cash flow (DCF) models are used to estimate the value of an investment based on its expected future cash flows. The cash flows calculated from this model are used by investors, investment bankers, research analysts and companies to make a variety of decisions.  Investors may use this model to make new investment decisions, while investment bankers can utilize it to value an asset. Research analysts may prepare this model to give stock recommendations, whereas companies can optimize the model to make efficient capital budgeting and operating expenditure decisions.

DCF analysis works on the concept of the time value of money. It can be used in a situation where someone is investing money in the present with expectations of receiving more money in the future. In simple terms, DCF models will help you calculate the present value of expected future cash flows using a discount rate. If these discounted cash flows over a period of the project are greater than the investment, a person may consider investing their money into the project. 

Let's begin! 

Import Dependencies

Please import the following libraries for the purpose of running this program smoothly - numpy, pandas and scipy. 
The above mentioned libraries can be installed in Jupyter notebook by typing the following in the Command prompt.
pip install numpy
pip install pandas
pip install scipy

In [71]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

Revenues

The first step is to import pricing strips from Excel. We will import crude oil, natural gas and NGL pricing strips from this excel file using Pandas module. The Excel File can be downloaded from here. 
Notice that this dataframe has four columns containing date, oil prices, gas prices and NGL prices. 

In [72]:
prices = pd.read_excel("Pricing_Oil_Gas_NGL.xlsx")
prices

,Date,Oil_Price,NGL_Price,Gas_Price
0,2014-08-01,100.0776,37.0720,4.446
1,2014-09-01,100.1052,37.1595,4.468
2,2014-10-01,99.2128,36.8970,4.457
3,2014-11-01,98.2284,36.5330,4.459
4,2014-12-01,97.2348,36.1795,4.496
...,...,...,...,...
288,2038-08-01,82.4412,30.3765,5.067
289,2038-09-01,82.4412,30.3765,5.067
290,2038-10-01,82.4412,30.3765,5.067
291,2038-11-01,82.4412,30.3765,5.067


Similarly, we will be reading another Excel file that carries the production numbers expected from this particular oil and gas well. This Excel file also has four columns containing date, oil production, gas production and NGL production. You can download the file from here.

In [73]:
production = pd.read_excel("Production_Oil_Gas_NGL.xlsx")
production

,Date,Oil_Prod,NGL_Prod,Gas_Prod
0,2014-08-01,16778.787645,1095.658543,5681.192447
1,2014-09-01,12654.595199,840.427184,4357.770582
2,2014-10-01,10261.173695,693.084143,3593.769631
3,2014-11-01,8682.106765,596.418602,3092.540899
4,2014-12-01,7555.366259,527.860023,2737.051973
...,...,...,...,...
288,2038-08-01,327.092701,31.769343,164.729925
289,2038-09-01,324.827784,31.549359,163.589271
290,2038-10-01,322.578545,31.330898,162.456508
291,2038-11-01,320.344894,31.113952,161.331605


Once we have the prices and production, the next step is to calculate revenues. For this purpose, I will merge the two dataframes on the date column. Once we have a combined dataframe, we will use a simple formula for calculating revenues i.e. price times production. For example, oil revenue is equal to oil price times oil production.

In [74]:
merged_data = pd.merge(prices, production, how='outer', on='Date')
merged_data['Oil_Revenue'] = merged_data['Oil_Price'] * merged_data['Oil_Prod']
merged_data['NGL_Revenue'] = merged_data['NGL_Price'] * merged_data['NGL_Prod']
merged_data['Gas_Revenue'] = merged_data['Gas_Price'] * merged_data['Gas_Prod']
merged_data

,Date,Oil_Price,NGL_Price,Gas_Price,Oil_Prod,NGL_Prod,Gas_Prod,Oil_Revenue,NGL_Revenue,Gas_Revenue
0,2014-08-01,100.0776,37.0720,4.446,16778.787645,1095.658543,5681.192447,1.679181e+06,40618.253520,25258.581620
1,2014-09-01,100.1052,37.1595,4.468,12654.595199,840.427184,4357.770582,1.266791e+06,31229.853933,19470.518961
2,2014-10-01,99.2128,36.8970,4.457,10261.173695,693.084143,3593.769631,1.018040e+06,25572.725629,16017.431246
3,2014-11-01,98.2284,36.5330,4.459,8682.106765,596.418602,3092.540899,8.528295e+05,21788.960787,13789.639870
4,2014-12-01,97.2348,36.1795,4.496,7555.366259,527.860023,2737.051973,7.346445e+05,19097.711716,12305.785671
...,...,...,...,...,...,...,...,...,...,...
288,2038-08-01,82.4412,30.3765,5.067,327.092701,31.769343,164.729925,2.696591e+04,965.041439,834.686531
289,2038-09-01,82.4412,30.3765,5.067,324.827784,31.549359,163.589271,2.677919e+04,958.359117,828.906837
290,2038-10-01,82.4412,30.3765,5.067,322.578545,31.330898,162.456508,2.659376e+04,951.723022,823.167126
291,2038-11-01,82.4412,30.3765,5.067,320.344894,31.113952,161.331605,2.640962e+04,945.132976,817.467243


Next, we will calculate the total revenues. However, in the oil and gas industry the company is obligated to pay revenue interest for land rights to landowners when drilling a well. The industry terminology is net revenue interest (NRI). In our case, the Net Revenue Interest is 75%. To account for 75% NRI, we will create three seperate columns that calculate revenues for oil, gas and NGL commodities, accordingly. 

Once we have the revenues from each stream, we can create a column in our dataframe to calculate the total revenue from this asset.

In [75]:
nri = 0.75 #Net Revenue Interest 
merged_data['Oil_Rev_NRI'] = merged_data['Oil_Price'] * merged_data['Oil_Prod']
merged_data['NGL_Rev_NRI'] = merged_data['NGL_Price'] * merged_data['NGL_Prod']
merged_data['Gas_Rev_NRI'] = merged_data['Gas_Price'] * merged_data['Gas_Prod']
merged_data['Total_Revenue'] = merged_data['Oil_Rev_NRI'] + merged_data['NGL_Rev_NRI'] + merged_data['Gas_Rev_NRI']
merged_data[['Date','Oil_Rev_NRI','NGL_Rev_NRI','Gas_Rev_NRI','Total_Revenue']]

,Date,Oil_Rev_NRI,NGL_Rev_NRI,Gas_Rev_NRI,Total_Revenue
0,2014-08-01,1.679181e+06,40618.253520,25258.581620,1.745058e+06
1,2014-09-01,1.266791e+06,31229.853933,19470.518961,1.317491e+06
2,2014-10-01,1.018040e+06,25572.725629,16017.431246,1.059630e+06
3,2014-11-01,8.528295e+05,21788.960787,13789.639870,8.884081e+05
4,2014-12-01,7.346445e+05,19097.711716,12305.785671,7.660480e+05
...,...,...,...,...,...
288,2038-08-01,2.696591e+04,965.041439,834.686531,2.876564e+04
289,2038-09-01,2.677919e+04,958.359117,828.906837,2.856646e+04
290,2038-10-01,2.659376e+04,951.723022,823.167126,2.836865e+04
291,2038-11-01,2.640962e+04,945.132976,817.467243,2.817222e+04


Taxes and Expenses

In the next section, we will be calculating different expenses and taxes that are inherent to exploration and production operation. We will be creating separate columns in our dataframe to calculate Lease Operating Expenses (LOE), Ad Valorem Tax and Severance Tax. The definition of these terms is given below:

LOE or Lease operating expenses are the costs incurred by an operator to keep production flowing after the initial cost of drilling and completing a well have been incurred.

Ad Valorem Tax is the oil and gas production tax that varies based on property tax in the district of production.

Severance Tax is the tax rate calculated on the the type of well and production. 

In [76]:
LOE = 9500
Ad_Valorem_Tax = 0.025
Severance_Tax_Oil = 0.046
Severance_Tax_NGL = 0.075
Severance_Tax_Gas = 0.075

merged_data['LOE'] = [LOE for i in range(293)]
merged_data['Ad_Valorem_Tax'] = [Ad_Valorem_Tax * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_Oil'] = [Severance_Tax_Oil * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_NGL'] = [Severance_Tax_NGL * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_Gas'] = [Severance_Tax_Gas * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data[['Date','LOE','Ad_Valorem_Tax','Sev_Tax_Oil','Sev_Tax_NGL','Sev_Tax_Gas']]

,Date,LOE,Ad_Valorem_Tax,Sev_Tax_Oil,Sev_Tax_NGL,Sev_Tax_Gas
0,2014-08-01,9500,43626.440838,80272.651142,130879.322515,130879.322515
1,2014-09-01,9500,32937.278905,60604.593186,98811.836716,98811.836716
2,2014-10-01,9500,26490.748262,48742.976803,79472.244787,79472.244787
3,2014-11-01,9500,22210.201421,40866.770614,66630.604263,66630.604263
4,2014-12-01,9500,19151.200612,35238.209126,57453.601835,57453.601835
...,...,...,...,...,...,...
288,2038-08-01,9500,719.141068,1323.219565,2157.423204,2157.423204
289,2038-09-01,9500,714.161456,1314.057079,2142.484368,2142.484368
290,2038-10-01,9500,709.216313,1304.958015,2127.648938,2127.648938
291,2038-11-01,9500,704.305443,1295.922014,2112.916328,2112.916328


CAPEX - Capital Expenditure

One of the most critical cost component of a DCF model is CAPEX or capital expenditure estimation. These are funds used by a company to drill and complete an oil and gas well. In our case, the CAPEX for drilling this well is approximately $6 million. Since, CAPEX is spent before any production can be obtained, this cost is captured in the 'zeroth' production month. Once we have all the revenue and cost components ready, we will simply subtract them to obtain cash flows. Using them, we can calculate cumulative cash flows to understand the model in much more detail. 

In [77]:
merged_data['D&CCAPEX'] = [0 for i in range(293)]
merged_data['D&CCAPEX'].iloc[0] = 6159101
merged_data['Cash Flow'] = merged_data['Total_Revenue'] - merged_data['D&CCAPEX'] - merged_data['Ad_Valorem_Tax'] - merged_data['Sev_Tax_Oil'] - merged_data['Sev_Tax_NGL'] - merged_data['Sev_Tax_Gas']
merged_data['Cumulative Cash Flows'] = np.cumsum(merged_data['Cash Flow'])
merged_data[['Date','D&CCAPEX','Cash Flow','Cumulative Cash Flows']]

c:\users\risha\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Date,D&CCAPEX,Cash Flow,Cumulative Cash Flows
0,2014-08-01,6159101,-4.799701e+06,-4.799701e+06
1,2014-09-01,0,1.026326e+06,-3.773375e+06
2,2014-10-01,0,8.254517e+05,-2.947924e+06
3,2014-11-01,0,6.920699e+05,-2.255854e+06
4,2014-12-01,0,5.967514e+05,-1.659102e+06
...,...,...,...,...
288,2038-08-01,0,2.240844e+04,2.589366e+07
289,2038-09-01,0,2.225327e+04,2.591591e+07
290,2038-10-01,0,2.209918e+04,2.593801e+07
291,2038-11-01,0,2.194616e+04,2.595996e+07


According to our model, the total cash flows from this oil and gas well are approximately $26 million. 

In [78]:
merged_data['Cash Flow'].sum()

25981749.764408834

NPV, IRR, Payback Period and MOIC

This next step is to calculate the NPV, IRR, Payback Period and MOIC of this model. But first, a quick recap of these four important financial metrics.

NPV or Net Present Value (NPV) is the value of all future cash flows (positive and negative) over the entire lifetime of a project. The cash flows used in NPV calculation are discounted to account for the time value of money & project risk.

IRR or Internal rate of return (IRR) is the discount rate at which the net present value of an investment is equal to zero. 

Payback Period is the amount of time it takes to recover the cost of an investment. 

MOIC or Multiple on Invested Capital allows investors to measure how much value an investment has generated i.e. a multiple of 2x means that the principal investment amount has increased in value by 100%.

I've created four functions in Python to calculate these financial indicators. One point to note is the use of fsolve from the SciPy library to calculate NPV and IRR.

x0 - The starting estimate for the roots of func(x) = 0 i.e. 10%
cashfs - Array of cash flow values computed by our model 
t - Array of values (months) computed specifically to calculate NPV and IRR

Calculation of MOIC and Payback Period are computed based on simple formulas that use cumulative cash flows and capital expenditure values.

In [79]:
cash_flow = merged_data['Cash Flow'].values
merged_data['Time'] = [i for i in range(293)]
t = merged_data['Time'].values

def npv(irr, cashfs, t):  
    x =  np.sum(cashfs / (1 + irr) ** t)
    return round(x, 3)

def irr(cashfs, t, x0, **kwargs):
    y = (fsolve(npv, x0=x0, args=(cashfs, t), **kwargs)).item()
    return round(y*100, 4)
    
def payback():
    final_full_year = merged_data[merged_data['Cumulative Cash Flows'] < 0].index.values.max()
    fractional_yr = -merged_data['Cumulative Cash Flows'][final_full_year]/merged_data['Cash Flow'][final_full_year + 1]
    period = final_full_year + fractional_yr
    return round(period, 1)

def moic():
    z = merged_data['Cash Flow'].sum()/merged_data['D&CCAPEX'].sum()
    return round(z,1)

print("NPV : $", npv(irr=0.10, cashfs=cash_flow, t=t))
print("IRR : ", irr(cashfs=cash_flow, t=t, x0=0.10), "%")
print("Payback Period : ", payback(), "Months")
print("MOIC : ", moic(), "x")

NPV : $ 710608.99
IRR :  12.1944 %
Payback Period :  7.0 Months
MOIC :  4.2 x


In the final step, we use a Pandas to convert our dataframe to a comma-separated values (csv) file using to_csv. 

In [ ]:
merged_data.to_csv("Model_Single_Well.csv")

Conclusion 

In this article, you learned how to use Python to create a financial model. We calculated the following metrics :-

- Net Present Value - NPV@10% : $710,609 
- Internal Rate of Return - IRR : 12.2%
- Payback Period : 7.0 Months
- Multiple on Invested Capital - MOIC : 4.2x
    
The full code can be found on my GitHub Page. I hope you enjoyed this step-by-step tutorial on how to create a simple financial model with Python. If you are interested in reading about my other posts, check out the links below. Happy coding!